In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install fastapi

     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 745 kB 47.8 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 10.9 MB 49.7 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=c0bd4c888ee09e7af26a0836690e34c144aacb320920af6974a55ce8d989cf2a
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [79]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
from keras.models import load_model

from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/home/{name}')
def home(name:str):
    return {'Welcome! ' + name}

@app.get('/modelpredictions/{modelInput}')
def getPredictions(modelInput:str):



    valueRange = {'Hygiene':'1-5',
                      'Micromanagement':'1-10',
                      'Working_Hours':'1-24',
                      'Salary':'100-50000',
                      'Age':'10-70',
                      'Infrastructure':'1-10',
                      'Food_Quality':'1-5',
                      'Incentives':'12-10000',
                      'Family_Size':'1-10',
                      'Experience':'1-60',
                      'Share_Price':'2-1500',
                      'Attrition':'1-40',
                      'Contract_Workers':'10-10000'
                      }

    paramMapping = {
                  'Hygiene':0,
                  'Micromanagement':1,
                  'Working_Hours':2,
                  'Salary':3,
                  'Age':4,
                  'Infrastructure':5,
                  'Food_Quality':6,
                  'Incentives':7,
                  'Family_Size':8,
                  'Experience':9,
                  'Share_Price':10,
                  'Attrition':11,
                  'Contract_Workers':12
                  }

    analyticParamsRange = {"Location_Score":"8.5-51.8",
                           "Internal_Audit":"6.44-47.84",
                           "External_Audit":"4.54-39.62",
                           "Fin_Score":"5.40-48.50",
                           "Loss_Score":"0.64-29.62",
                           "Past_Score":"3.68-47.75"}


    locations = {'Asia_Pacific': 51.8,
             'Developed Economies and EU':46.9,
             'Central and South-Eastern Europe and CIS':18.0,
             'Africa':13.1,
             'Latin America and the Caribbean':12.0,
             'Middle East':8.5
            }

    dfReturn = []
    modelPredict = []

    splitRows = modelInput.split('@')


    splitInputs = modelInput.split('-')[1:]

    modelInputsAll = []


    for row in splitRows:

        splitInputs = row.split('-')[1:]
        df = []
        modelInputs = []

        for val in splitInputs:
            modelInputs.append(float(val))

        modelInputsAll.append(modelInputs)

    
    
        location_score = locations[row.split('-')[0]]

        internal_audit = 0.2*(modelInputs[paramMapping['Hygiene']] + 
                              modelInputs[paramMapping['Micromanagement']] + 
                              modelInputs[paramMapping['Working_Hours']] + 
                              modelInputs[paramMapping['Salary']] +
                              modelInputs[paramMapping['Age']])
        
        external_audit = 0.2*(modelInputs[paramMapping['Infrastructure']] + 
                              modelInputs[paramMapping['Food_Quality']] + 
                              modelInputs[paramMapping['Incentives']] + 
                              modelInputs[paramMapping['Age']])
        
        fin_score = 0.2*(modelInputs[paramMapping['Salary']] + 
                              modelInputs[paramMapping['Family_Size']] + 
                              modelInputs[paramMapping['Working_Hours']] + 
                              modelInputs[paramMapping['Age']] +
                              modelInputs[paramMapping['Experience']])
        
        loss_score = 0.2*(modelInputs[paramMapping['Share_Price']] + 
                              modelInputs[paramMapping['Attrition']] + 
                              modelInputs[paramMapping['Contract_Workers']])
        
        past_score = 0.2*(modelInputs[paramMapping['Hygiene']] + 
                              modelInputs[paramMapping['Infrastructure']] + 
                              modelInputs[paramMapping['Salary']] + 
                              modelInputs[paramMapping['Share_Price']] +
                              modelInputs[paramMapping['Attrition']])

        df.append(location_score)
        df.append(internal_audit)
        df.append(external_audit)
        df.append(fin_score)
        df.append(loss_score)
        df.append(past_score)
        

        for k in valueRange:

            minVal, maxVal = valueRange[k].split('-')
            minVal = float(minVal)
            maxVal = float(maxVal)
            modelInputs[paramMapping[k]] = (modelInputs[paramMapping[k]]-minVal)/(maxVal-minVal)
        
        minVal, maxVal = analyticParamsRange['Location_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Internal_Audit'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['External_Audit'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Fin_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Loss_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Past_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        modelPredict.append(modelInputs)
        dfReturn.append(df)

    model = load_model('/content/drive/MyDrive/ForcedLabour/fed-forced_labour.h5')
    modelPredict = np.array(modelPredict)
    modelPredict = modelPredict.reshape((len(splitRows),19))
    print("Sumit")
    print(modelPredict.shape)
    out = model.predict(modelPredict)
    print(out.shape)
    ReturnString = str(dfReturn[0][0]) + '-' + str(dfReturn[0][1]) + '-' + str(dfReturn[0][2]) + '-' + str(dfReturn[0][3]) + '-' + str(dfReturn[0][4]) + '-' + str(dfReturn[0][5])

    if out[0]<0.5:
            ReturnString = ReturnString + "Good"
    else:
        ReturnString = ReturnString + "Bad"

    for i in range(1, len(dfReturn)):

        ReturnString = ReturnString + "@" + str(dfReturn[i][0]) + '-' + str(dfReturn[i][1]) + '-' + str(dfReturn[i][2]) + '-' + str(dfReturn[i][3]) + '-' + str(dfReturn[i][4]) + '-' + str(dfReturn[i][5])

        if out[i]<0.5:
            ReturnString = ReturnString + "Good"
        else:
            ReturnString = ReturnString + "Bad"

    return ReturnString

@app.get('/')
def home():
    return({"key":"hello world"})

In [80]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://ecda-35-186-179-60.ngrok.io


INFO:     Started server process [65]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     157.50.12.204:0 - "GET / HTTP/1.1" 200 OK
INFO:     157.50.12.204:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     157.50.12.204:0 - "GET /modelpredictions/ HTTP/1.1" 404 Not Found
INFO:     157.50.12.204:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
Sumit
(1, 19)
(1, 1)
INFO:     157.50.12.204:0 - "GET /modelpredictions/Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1 HTTP/1.1" 200 OK
Sumit
(2, 19)
(2, 1)
INFO:     157.50.12.204:0 - "GET /modelpredictions/Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1%40Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1 HTTP/1.1" 200 OK
Sumit
(5, 19)
(5, 1)
INFO:     157.50.12.204:0 - "GET /modelpredictions/Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1%40Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1%40Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1%40Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1%40Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [65]


In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [504]
INFO:     Waiting for application startup.


In [54]:
import pandas as pd
import numpy as np
from keras.models import load_model

def getPredictionsSingle(modelInput:str):

    splitInputs = modelInput.split('-')[1:]

    modelInputs = []

    for val in splitInputs:
        modelInputs.append(float(val))

    valueRange = {'Hygiene':'1-5',
                  'Micromanagement':'1-10',
                  'Working_Hours':'1-24',
                  'Salary':'100-50000',
                  'Age':'10-70',
                  'Infrastructure':'1-10',
                  'Food_Quality':'1-5',
                  'Incentives':'12-10000',
                  'Family_Size':'1-10',
                  'Experience':'1-60',
                  'Share_Price':'2-1500',
                  'Attrition':'1-40',
                  'Contract_Workers':'10-10000'
                  }

    paramMapping = {
                  'Hygiene':0,
                  'Micromanagement':1,
                  'Working_Hours':2,
                  'Salary':3,
                  'Age':4,
                  'Infrastructure':5,
                  'Food_Quality':6,
                  'Incentives':7,
                  'Family_Size':8,
                  'Experience':9,
                  'Share_Price':10,
                  'Attrition':11,
                  'Contract_Workers':12
                  }

    analyticParamsRange = {"Location_Score":"8.5-51.8",
                           "Internal_Audit":"6.44-47.84",
                           "External_Audit":"4.54-39.62",
                           "Fin_Score":"5.40-48.50",
                           "Loss_Score":"0.64-29.62",
                           "Past_Score":"3.68-47.75"}


    locations = {'Asia_Pacific': 51.8,
             'Developed Economies and EU':46.9,
             'Central and South-Eastern Europe and CIS':18.0,
             'Africa':13.1,
             'Latin America and the Caribbean':12.0,
             'Middle East':8.5
            }
    
    location_score = locations[modelInput.split('-')[0]]

    internal_audit = 0.2*(modelInputs[paramMapping['Hygiene']] + 
                          modelInputs[paramMapping['Micromanagement']] + 
                          modelInputs[paramMapping['Working_Hours']] + 
                          modelInputs[paramMapping['Salary']] +
                          modelInputs[paramMapping['Age']])
    
    external_audit = 0.2*(modelInputs[paramMapping['Infrastructure']] + 
                          modelInputs[paramMapping['Food_Quality']] + 
                          modelInputs[paramMapping['Incentives']] + 
                          modelInputs[paramMapping['Age']])
    
    fin_score = 0.2*(modelInputs[paramMapping['Salary']] + 
                          modelInputs[paramMapping['Family_Size']] + 
                          modelInputs[paramMapping['Working_Hours']] + 
                          modelInputs[paramMapping['Age']] +
                          modelInputs[paramMapping['Experience']])
    
    loss_score = 0.2*(modelInputs[paramMapping['Share_Price']] + 
                          modelInputs[paramMapping['Attrition']] + 
                          modelInputs[paramMapping['Contract_Workers']])
    
    past_score = 0.2*(modelInputs[paramMapping['Hygiene']] + 
                          modelInputs[paramMapping['Infrastructure']] + 
                          modelInputs[paramMapping['Salary']] + 
                          modelInputs[paramMapping['Share_Price']] +
                          modelInputs[paramMapping['Attrition']])


    

    for k in valueRange:
        print(paramMapping[k])
        print(modelInputs[paramMapping[k]])
        minVal, maxVal = valueRange[k].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs[paramMapping[k]] = (modelInputs[paramMapping[k]]-minVal)/(maxVal-minVal)
    
    minVal, maxVal = analyticParamsRange['Location_Score'].split('-')
    minVal = float(minVal)
    maxVal = float(maxVal)
    modelInputs.append((location_score-minVal)/(maxVal-minVal))

    minVal, maxVal = analyticParamsRange['Internal_Audit'].split('-')
    minVal = float(minVal)
    maxVal = float(maxVal)
    modelInputs.append((location_score-minVal)/(maxVal-minVal))

    minVal, maxVal = analyticParamsRange['External_Audit'].split('-')
    minVal = float(minVal)
    maxVal = float(maxVal)
    modelInputs.append((location_score-minVal)/(maxVal-minVal))

    minVal, maxVal = analyticParamsRange['Fin_Score'].split('-')
    minVal = float(minVal)
    maxVal = float(maxVal)
    modelInputs.append((location_score-minVal)/(maxVal-minVal))

    minVal, maxVal = analyticParamsRange['Loss_Score'].split('-')
    minVal = float(minVal)
    maxVal = float(maxVal)
    modelInputs.append((location_score-minVal)/(maxVal-minVal))

    minVal, maxVal = analyticParamsRange['Past_Score'].split('-')
    minVal = float(minVal)
    maxVal = float(maxVal)
    modelInputs.append((location_score-minVal)/(maxVal-minVal))



    model = load_model('/content/drive/MyDrive/ForcedLabour/fed-forced_labour.h5')
    modelInputs = np.array(modelInputs)
    modelInputs = modelInputs.reshape((1,19))
    print(modelInputs.shape)
    out = model.predict(modelInputs)[0][0]

    s = str(location_score) + '-' + str(internal_audit) + '-' + str(external_audit) + '-' + str(fin_score) + '-' + str(loss_score) + '-' + str(past_score)

    if out<0.5:
      return s+ "-Good"
    else:
      return s+ "-Bad"

In [74]:
def getPredictionsAll(modelInput:str):



    valueRange = {'Hygiene':'1-5',
                      'Micromanagement':'1-10',
                      'Working_Hours':'1-24',
                      'Salary':'100-50000',
                      'Age':'10-70',
                      'Infrastructure':'1-10',
                      'Food_Quality':'1-5',
                      'Incentives':'12-10000',
                      'Family_Size':'1-10',
                      'Experience':'1-60',
                      'Share_Price':'2-1500',
                      'Attrition':'1-40',
                      'Contract_Workers':'10-10000'
                      }

    paramMapping = {
                  'Hygiene':0,
                  'Micromanagement':1,
                  'Working_Hours':2,
                  'Salary':3,
                  'Age':4,
                  'Infrastructure':5,
                  'Food_Quality':6,
                  'Incentives':7,
                  'Family_Size':8,
                  'Experience':9,
                  'Share_Price':10,
                  'Attrition':11,
                  'Contract_Workers':12
                  }

    analyticParamsRange = {"Location_Score":"8.5-51.8",
                           "Internal_Audit":"6.44-47.84",
                           "External_Audit":"4.54-39.62",
                           "Fin_Score":"5.40-48.50",
                           "Loss_Score":"0.64-29.62",
                           "Past_Score":"3.68-47.75"}


    locations = {'Asia_Pacific': 51.8,
             'Developed Economies and EU':46.9,
             'Central and South-Eastern Europe and CIS':18.0,
             'Africa':13.1,
             'Latin America and the Caribbean':12.0,
             'Middle East':8.5
            }

    dfReturn = []
    modelPredict = []

    splitRows = modelInput.split('@')


    splitInputs = modelInput.split('-')[1:]

    modelInputsAll = []


    for row in splitRows:

        splitInputs = row.split('-')[1:]
        df = []
        modelInputs = []

        for val in splitInputs:
            modelInputs.append(float(val))

        modelInputsAll.append(modelInputs)

    
    
        location_score = locations[row.split('-')[0]]

        internal_audit = 0.2*(modelInputs[paramMapping['Hygiene']] + 
                              modelInputs[paramMapping['Micromanagement']] + 
                              modelInputs[paramMapping['Working_Hours']] + 
                              modelInputs[paramMapping['Salary']] +
                              modelInputs[paramMapping['Age']])
        
        external_audit = 0.2*(modelInputs[paramMapping['Infrastructure']] + 
                              modelInputs[paramMapping['Food_Quality']] + 
                              modelInputs[paramMapping['Incentives']] + 
                              modelInputs[paramMapping['Age']])
        
        fin_score = 0.2*(modelInputs[paramMapping['Salary']] + 
                              modelInputs[paramMapping['Family_Size']] + 
                              modelInputs[paramMapping['Working_Hours']] + 
                              modelInputs[paramMapping['Age']] +
                              modelInputs[paramMapping['Experience']])
        
        loss_score = 0.2*(modelInputs[paramMapping['Share_Price']] + 
                              modelInputs[paramMapping['Attrition']] + 
                              modelInputs[paramMapping['Contract_Workers']])
        
        past_score = 0.2*(modelInputs[paramMapping['Hygiene']] + 
                              modelInputs[paramMapping['Infrastructure']] + 
                              modelInputs[paramMapping['Salary']] + 
                              modelInputs[paramMapping['Share_Price']] +
                              modelInputs[paramMapping['Attrition']])

        df.append(location_score)
        df.append(internal_audit)
        df.append(external_audit)
        df.append(fin_score)
        df.append(loss_score)
        df.append(past_score)
        

        for k in valueRange:

            minVal, maxVal = valueRange[k].split('-')
            minVal = float(minVal)
            maxVal = float(maxVal)
            modelInputs[paramMapping[k]] = (modelInputs[paramMapping[k]]-minVal)/(maxVal-minVal)
        
        minVal, maxVal = analyticParamsRange['Location_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Internal_Audit'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['External_Audit'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Fin_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Loss_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        minVal, maxVal = analyticParamsRange['Past_Score'].split('-')
        minVal = float(minVal)
        maxVal = float(maxVal)
        modelInputs.append((location_score-minVal)/(maxVal-minVal))

        modelPredict.append(modelInputs)
        dfReturn.append(df)

    model = load_model('/content/drive/MyDrive/ForcedLabour/fed-forced_labour.h5')
    modelPredict = np.array(modelPredict)
    modelPredict = modelPredict.reshape((len(splitRows),19))
    print("Sumit")
    print(modelPredict.shape)
    out = model.predict(modelPredict)
    print(out.shape)
    ReturnString = str(dfReturn[0][0]) + '-' + str(dfReturn[0][1]) + '-' + str(dfReturn[0][2]) + '-' + str(dfReturn[0][3]) + '-' + str(dfReturn[0][4]) + '-' + str(dfReturn[0][5])

    if out[0]<0.5:
            ReturnString = ReturnString + "Good"
    else:
        ReturnString = ReturnString + "Bad"

    for i in range(1, len(dfReturn)):

        ReturnString = ReturnString + "@" + str(dfReturn[i][0]) + '-' + str(dfReturn[i][1]) + '-' + str(dfReturn[i][2]) + '-' + str(dfReturn[i][3]) + '-' + str(dfReturn[i][4]) + '-' + str(dfReturn[i][5])

        if out[i]<0.5:
            ReturnString = ReturnString + "Good"
        else:
            ReturnString = ReturnString + "Bad"

    return ReturnString

In [76]:
s = "Asia_Pacific-1-1-1-1-1-1-1-1-1-1-1-1-1"

print(getPredictionsSingle(s))
print(getPredictionsAll(s))

0
1.0
1
1.0
2
1.0
3
1.0
4
1.0
5
1.0
6
1.0
7
1.0
8
1.0
9
1.0
10
1.0
11
1.0
12
1.0
(1, 19)
51.8-1.0-0.8-1.0-0.6000000000000001-1.0-Good
Sumit
(1, 19)
(1, 1)
51.8-1.0-0.8-1.0-0.6000000000000001-1.0Good
